In [1]:
from pynq import Overlay
design_cdma = Overlay("./hw4.bit")

/usr/local/lib/python3.6/dist-packages/pynq/overlay.py:299: UserWarning: Users will not get PARAMETERS / REGISTERS information through TCL files. HWH file is recommended.
  warnings.warn(message, UserWarning)


In [2]:
design_cdma.ip_dict

{'axi_cdma_0': {'addr_range': 65536,
  'driver': pynq.overlay.DefaultIP,
  'fullpath': 'axi_cdma_0',
  'gpio': {},
  'interrupts': {},
  'mem_id': 'SEG_axi_cdma_0_Reg',
  'phys_addr': 2116026368,
  'state': None,
  'type': 'xilinx.com:ip:axi_cdma:4.1'}}

In [3]:
address_cdma = design_cdma.ip_dict['axi_cdma_0']['phys_addr']
print("Physical address of cdma: 0x" + format(address_cdma, '02x'))

Physical address of cdma: 0x7e200000


In [4]:
address_zynq = 0x30000000

In [5]:
from pynq import MMIO
cdma = MMIO(address_cdma, 0x30)
zynq = MMIO(address_zynq, 0x30)

In [6]:
#Step1: Put two numbers into the system's memory (at address + 0 and address + 4)
a = 60 #can be edited
b = 80 #can be edited
zynq.write(0x0, a)
zynq.write(0x4, b)

#Step2: Set CDMA to move data from ZYNQ to block memory
cdma.write(0x0, 0x04) #CDMACR (CDMA_control)
cdma.write(0x18, address_zynq) #SA (Source Address) 
cdma.write(0x20, 0xc0000000) #DA (Destination Address) 
cdma.write(0x28, 0x8) #BTT (the number of Bytes to be transmitted)

#Step3: Set CDMA to move the data back from the block memory to ZYNQ
cdma.write(0x0, 0x04) #CDMACR
cdma.write(0x18, 0xc0000000) #SA
cdma.write(0x20, address_zynq) #DA
cdma.write(0x28, 0x10) #BTT

In [7]:
print(f"ans of {a}*{b} = {zynq.read(0x000c)}") #Read the data of ZYNQ address + 0x0C and get the answer

ans of 60*80 = 4800
